# Setup

In [1]:
from sqlmodel import SQLModel, create_engine, Field, Session
from typing import Optional, List, Tuple, Any
from datetime import datetime, timezone, timedelta
import enum

#sqlite_url = "sqlite:///:memory:"
sqlite_url = "sqlite:///TEST.db"
engine = create_engine(sqlite_url)

In [2]:
def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

class Asset(str, enum.Enum):
    btcusd = "btcusd"
    ethusd = "ethusd"

class Action(int, enum.Enum):
    SELL = 0
    BUY = 1

class Tick(SQLModel, table=True):
    date: datetime = Field(primary_key=True, nullable=False)
    asset: Asset = Field(primary_key=True, nullable=False)
    o: float = Field(nullable=False)
    h: float = Field(nullable=False)
    l: float = Field(nullable=False)
    c: float = Field(nullable=False)
    v: float = Field(nullable=False)

class Signal(SQLModel, table=True):
    date: datetime = Field(primary_key=True, nullable=False)
    asset: Asset = Field(primary_key=True, nullable=False)
    probability: float = Field(nullable=False)
    threshold: float = Field(nullable=False)
    action: Action = Field(nullable=False)

create_db_and_tables()

# Main Execution

In [3]:
now = datetime.now()

tick1: Tick = Tick(
    date=now
    , asset=Asset.btcusd
    , o=1
    , h=2
    , l=3
    , c=4
    , v=5
)

signal1: Signal = Signal(
    date=now
    , asset=Asset.btcusd
    , probability=0.58
    , threshold=0.5
    , action=1
)

signal2: Signal = Signal(
    date=now
    , asset=Asset.ethusd
    , probability=0.75
    , threshold=0.5
    , action=1
)


In [9]:
with Session(engine) as session:
    session.add(tick1)
    session.add(signal1)
    session.add(signal2)
    session.commit()
    session.refresh(tick1)
    session.refresh(signal1)
    session.refresh(signal2)
print(signal1)
print()
print(signal2)


action=<Action.BUY: 1> threshold=0.5 date=datetime.datetime(2024, 2, 29, 14, 16, 21, 970615) probability=0.58 asset=<Asset.btcusd: 'btcusd'>

action=<Action.BUY: 1> threshold=0.5 date=datetime.datetime(2024, 2, 29, 14, 16, 21, 970615) probability=0.75 asset=<Asset.ethusd: 'ethusd'>


In [11]:
signal3: Signal = Signal(
    date=now
    , asset=Asset.ethusd
    , probability=0.75
    , threshold=0.95
    , action=0
)

try:
    with Session(engine) as session:
        session.add(signal3)
        session.commit()
        session.refresh(signal3)
        print(signal3)
except:
    print('failed to commit signal3')


failed to commit signal3


# Is `Classify_Future()` accurate?


In [6]:
import pandas as pd
import numpy as np

def classify_future(
    future_deltas
    , max_trailing_loss=0.10
    , min_expected_return=0.03
):
    trailing_loss = 0.0
    pnl = 0.0
    future_profiltable = False
    stopped_out = False
    trade_duration = 0
    for delta in future_deltas:
        trade_duration += 1
        if trade_duration == 1:
            pnl = 1 + delta
            trailing_loss = delta
        else:
            new_pnl = pnl * (1 + delta)
            trailing_loss = trailing_loss + (new_pnl - pnl)
            pnl = new_pnl
        if trailing_loss >= 0.0:
            trailing_loss = 0.0
        if abs(trailing_loss) >= abs(max_trailing_loss):
            # Failed trade, sell position, stop trade
            stopped_out = True
            break
    pnl = pnl - 1
    future_profitable = pnl >= min_expected_return
    return future_profitable, trade_duration, pnl, trailing_loss


future_deltas = np.array([0.03 , 0.025, 0.05 , 0.04 , 0.03 , 0.03 , 0.025, -0.05 , 0.04 ,0.03 , 0.03 , -0.025, 0.05 , 0.04 , 0.03 , 0.03 , 0.025, 0.05 , -0.04 , 0.03 ])
future_deltas, len(future_deltas)

(array([ 0.03 ,  0.025,  0.05 ,  0.04 ,  0.03 ,  0.03 ,  0.025, -0.05 ,
         0.04 ,  0.03 ,  0.03 , -0.025,  0.05 ,  0.04 ,  0.03 ,  0.03 ,
         0.025,  0.05 , -0.04 ,  0.03 ]),
 20)

In [10]:
profitable, duration, pnl, drawdown = classify_future(future_deltas)

print(f"Profitable: {profitable}, Duration: {duration:.3f}, Profit: {pnl:.3f}, Trailing Loss: {drawdown:.3f}")


Profitable: True, Duration: 20.000, Profit: 0.580, Trailing Loss: -0.018


In [7]:
r_simple = np.sum(future_deltas)
r_compound = np.prod(future_deltas + 1) - 1

print(f"Simple Return: {r_simple:.3f} \nCompound Return: {r_compound:.3f}")

Simple Return: 0.470 
Compound Return: 0.580


In [8]:
future2 = np.concatenate([future_deltas,future_deltas])

r_simple = np.sum(future2)
r_compound = np.prod(future2 + 1) - 1

print(f"Simple Return: {r_simple:.3f} \nCompound Return: {r_compound:.3f}")

Simple Return: 0.940 
Compound Return: 1.495


# Updated Classify_Future()

In [13]:
def classify_future_new(
    future_deltas
    , max_trailing_loss=0.10
    , min_expected_return=0.03
):
    trailing_loss = 0.0
    pnl = 0.0
    future_profiltable = False
    stopped_out = False
    initial_value = 1
    current_value = 1
    trade_duration = 0
    for delta in future_deltas:
        trade_duration += 1
        pnl = (1 + pnl) * (1 + delta) - 1
        current_value = current_value * (1 + pnl)
        trailing_loss = (current_value - initial_value)/initial_value
        if trailing_loss >= 0.0:
            trailing_loss = 0.0
        if abs(trailing_loss) >= abs(max_trailing_loss):
            # Failed trade, sell position, stop trade
            stopped_out = True
            break
    future_profitable = pnl >= min_expected_return
    return future_profitable, trade_duration, pnl, trailing_loss


In [14]:
profitable, duration, pnl, drawdown = classify_future_new(future_deltas)

print(f"Profitable: {profitable}, Duration: {duration:.3f}, Profit: {pnl:.3f}, Trailing Loss: {drawdown:.3f}")


Profitable: True, Duration: 20.000, Profit: 0.580, Trailing Loss: 0.000


# Try new Function on real data

In [15]:
from utils import process_ticks, compute_latest_stats, generate_timesteps_from_ticks, predict_via_serving
from utils import date_range, interval_infill, get_observation_v2, column_normalize, group_normalize, round_threshold

from db import DBManager, Asset, Tick, Timestep, ENVIRONMENT, MANAGER_ERROR
from hyperparameters import Params

from datetime import datetime, timedelta, timezone
from typing import List, Tuple, Any

model_endpoint = "http://192.168.50.61:8501/v1/models/model:predict"
acceptable_response_codes = [200]

params: Params = Params()
manager: DBManager = DBManager(environment=ENVIRONMENT.PREPROD)
params

Params(
	max_holding_period: 1,
	max_look_back_period: 18, 
	volatility_period: 7 
	min_expected_return: 0.03, 
	max_trailing_loss: 0.1 
	smas: {'s14': 672, 's50': 2400, 's100': 4800, 's350': 16800, 's700': 33600} 
	target_fields: ['c', 's50', 's100', 's350', 's700', 'v', 'hv'] 
	observation_shape: (24, 36, 7),
	interval: 30
	holding_period_in_T: 48,
	look_back_period_in_T: 864,
	volatility_period_in_T: 336,
)

In [16]:
start_date = datetime(2024, 1, 1, 17, 30, tzinfo=timezone.utc)
inference_frame_start_date = start_date - timedelta(days=18)
increment = timedelta(minutes=30)

max_timesteps = 5000
msg, timesteps = manager.get_historical_timesteps(start=inference_frame_start_date, frame_length=max_timesteps, asset=Asset.btcusd)

# Save Notebook

### Used from ipython to save the current execution history

---

In [ ]:
%notebook myhistory.ipynb